In [ ]:
# V2 :
# I encountered a problem with the submission merging deBERTa and roBERTa
# This version fix the issue
# _matrix_postprocess_location : added matrix_usage in recursive call
# _custom_weighted_box_fusion : deleted the wbf_location replacement

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
BASE_PATH = '..'

In [ ]:
sample_submission = pd.read_csv(f'{BASE_PATH}/input/nbme-score-clinical-patient-notes/sample_submission.csv')

In [ ]:
sample_submission

In [ ]:
other_submission = sample_submission.copy()
other_submission['location'] = [' 56 67', '223 245', '205 245;300 400', '', '0 3;4 5;6 10;34 45;56 60;11 14;74 102;103 111']

In [ ]:
other_submission

In [ ]:
def _trim_space_start_or_end(location_string):
    trim_location_string = location_string
    if not location_string != location_string:
        if location_string[0] == ' ' and location_string[-1] == ' ':
            trim_location_string = location_string[1:]
            trim_location_string = trim_location_string[:-1]
        elif location_string[0] == ' ':
            trim_location_string = location_string[1:]
        elif location_string[-1] == ' ':
            trim_location_string = location_string[:-1]
    return trim_location_string

In [ ]:
_trim_space_start_or_end(' 35 40 ')

In [ ]:
_trim_space_start_or_end('35 40 ')

In [ ]:
_trim_space_start_or_end(np.nan)

In [ ]:
def _get_start_end_from_location_string(base_location, other_location):
    base_location = _trim_space_start_or_end(base_location)
    other_location = _trim_space_start_or_end(other_location)
    base_start = int(base_location.split(' ')[0])
    base_end = int(base_location.split(' ')[1])
    other_start = int(other_location.split(' ')[0])
    other_end = int(other_location.split(' ')[1])
    return base_start, base_end, other_start, other_end

In [ ]:
_get_start_end_from_location_string('35 40', '45 50')

In [ ]:
def _get_location_difference(base_location, other_location):
    base_start, base_end, other_start, other_end = _get_start_end_from_location_string(base_location, other_location)
    start_difference = abs(base_start - other_start)
    end_difference = abs(base_end - other_end)
    mid_difference_base = abs(base_end - other_start)
    mid_difference_other = abs(other_end - base_start)
    return start_difference, end_difference, mid_difference_base, mid_difference_other

In [ ]:
_get_location_difference('35 40', '45 50')

In [ ]:
def _get_box_fusion(base_location, other_location):
    base_start, base_end, other_start, other_end = _get_start_end_from_location_string(base_location, other_location)
    mean_start = int((base_start + other_start) / 2)
    mean_end = int((base_end + other_end) / 2)
    box_fusion = f'{mean_start} {mean_end}'
    return box_fusion

In [ ]:
_get_box_fusion('35 40', '45 50')

In [ ]:
def _get_matrix_cross_difference(from_location_list):
    len_location_split = len(from_location_list)
    matrix_cross_location = np.ones((len_location_split, len_location_split), dtype='int') * 999
    for index_i in range(len_location_split):
        for index_j in range(index_i+1, len_location_split):
            location_difference = _get_location_difference(from_location_list[index_i], from_location_list[index_j])
            matrix_cross_location[index_i, index_j] = min(location_difference[2], location_difference[3])
    return matrix_cross_location

In [ ]:
_get_matrix_cross_difference(['0 3', '4 6', '10 23', '25 27'])

In [ ]:
def _get_matrix_start_position_difference(from_location_list):
    len_location_split = len(from_location_list)
    matrix_start_position_location = np.ones((len_location_split, len_location_split), dtype='int') * 999
    for index_i in range(len_location_split):
        for index_j in range(index_i+1, len_location_split):
            location_difference = _get_location_difference(from_location_list[index_i], from_location_list[index_j])
            matrix_start_position_location[index_i, index_j] = min(location_difference[0], location_difference[1])
    return matrix_start_position_location

In [ ]:
_get_matrix_start_position_difference(['0 3', '4 6', '10 23', '25 27'])

In [ ]:
def _order_by_start_position(location_list):
    tuple_list = [(int(element.split(' ')[0]), int(element.split(' ')[1])) for element in location_list]
    sorted_tuple_list = sorted(tuple_list, key=lambda t: t[0])
    ordered_location_list = [f"{element[0]} {element[1]}" for element in sorted_tuple_list]
    return ordered_location_list

In [ ]:
_order_by_start_position(['4 6', '0 3', '25 27', '10 23'])

In [ ]:
def _matrix_postprocess_location(location_string, matrix_usage='cross'):
    postprocess_location = location_string
    if location_string == '':
        postprocess_location = float('NaN')
    elif not location_string != location_string:
        location_split = location_string.split(';')
        location_split = [_trim_space_start_or_end(element) for element in location_split]
        location_split = list(set(location_split))
        location_split = _order_by_start_position(location_split)
        if len(location_split) > 1:
            if matrix_usage == 'cross':
                matrix_location = _get_matrix_cross_difference(location_split)
                where_ones = np.where(matrix_location <= 2)
            elif matrix_usage == 'start':
                matrix_location = _get_matrix_start_position_difference(location_split)
                where_ones = np.where(matrix_location <= 5)

            array_ones_size = where_ones[0].size
            if array_ones_size > 0:
                sub_location_split = []
                for index in range(array_ones_size):
                    index_i = where_ones[0][index]
                    index_j = where_ones[1][index]
                    start_end_location = _get_start_end_from_location_string(location_split[index_i], 
                                                                             location_split[index_j])
                    min_start = min(start_end_location[0], start_end_location[2])  
                    max_end = max(start_end_location[1], start_end_location[3])
                    sub_location_split.append(f'{min_start} {max_end}')
                
                processed_index = list(set(list(np.unique(where_ones[0])) + list(np.unique(where_ones[1]))))
                not_processed_index = [index_raw for index_raw in range(len(location_split)) if index_raw not in processed_index]
                for index_raw in not_processed_index:
                    sub_location_split.append(location_split[index_raw])
                sub_location = ';'.join(sub_location_split)
                postprocess_location = _matrix_postprocess_location(sub_location, matrix_usage=matrix_usage)
            else:
                postprocess_location = ';'.join(location_split)
    return postprocess_location

In [ ]:
_matrix_postprocess_location(np.nan)

In [ ]:
_matrix_postprocess_location('')

In [ ]:
_matrix_postprocess_location('0 3')

In [ ]:
_matrix_postprocess_location('0 3;4 5;6 10;34 45;56 60;11 14;74 102;103 111')
# Next function fix the duplicates from location that start or end +/- 3 position close
# e.g. 0 14 and 0 10

In [ ]:
def _postprocess_location(location_string):
    cross_matrix_postprocess = _matrix_postprocess_location(location_string)
    postprocess_location = _matrix_postprocess_location(cross_matrix_postprocess, matrix_usage='start')
    return postprocess_location

In [ ]:
_postprocess_location('0 3;4 5;6 10;34 45;56 60;11 14;74 102;103 111')

In [ ]:
_postprocess_location('')

In [ ]:
_postprocess_location('200 250;205 245;300 400')

In [ ]:
_postprocess_location('223 226;228 229;231 232;234 235;237 240;242 246;248 249;251 254;256 258')

In [ ]:
def _custom_weighted_box_fusion(base_location, other_location, treshold:int = 5):
    # Check if NaN (works with string and float('NaN') values)
    isnan_base = base_location != base_location
    isnan_other = other_location != other_location
    # If base is NaN and other is not, fill base value with other's
    if isnan_base and not isnan_other:
        wbf_location = other_location
    elif not isnan_other and not isnan_base:
        if base_location != other_location:
            wbf_location = ''
            base_location_split = base_location.split(";")
            other_location_split = other_location.split(';')
            set_location_split = list(set(base_location_split + other_location_split))
            first_set_location = set_location_split.pop(0)
            box_fusion_used = False
            for set_location_string in set_location_split:
                start_difference, end_difference, mid_difference_base, mid_difference_other = _get_location_difference(first_set_location, 
                                                                                                                       set_location_string)
                if start_difference <= treshold and end_difference <= treshold:
                    if wbf_location != '' and wbf_location[-1] != ';':
                        wbf_location += ';'
                    wbf_location += _get_box_fusion(first_set_location, set_location_string)
                    wbf_location += ';'
                    box_fusion_used = True
                else:
                    if wbf_location != '' and wbf_location[-1] != ';':
                        wbf_location += ';'
                    wbf_location += f'{first_set_location};{set_location_string}'

            if box_fusion_used:
                if wbf_location[-1] == ';':
                    wbf_location = wbf_location[:-1]
        else:
            wbf_location = base_location
    else:
        wbf_location = base_location
    
    return_wbf = _postprocess_location(wbf_location)     
    return return_wbf

In [ ]:
_custom_weighted_box_fusion('70 91', '71 91;177 183')

In [ ]:
_custom_weighted_box_fusion('696 724', '697 724')

In [ ]:
_custom_weighted_box_fusion('0 100', '223 245')

In [ ]:
_custom_weighted_box_fusion('0 100', '4 97')

In [ ]:
_custom_weighted_box_fusion('0 100', '10 87', treshold=15)

In [ ]:
def _check_duplications(location_string):
    no_duplicate = location_string
    if not location_string != location_string:
        location_split = location_string.split(';')
        location_split = [_trim_space_start_or_end(element) for element in location_split]
        location_split = list(set(location_split))
        location_split = _order_by_start_position(location_split)
        no_duplicate = ';'.join(location_split)
    return no_duplicate

In [ ]:
_check_duplications('15 20;0 10;0 10')

In [ ]:
postprocess_base_value = list(map(_postprocess_location, sample_submission.location.values))
print(sample_submission.location.values)
print(postprocess_base_value)

In [ ]:
postprocess_other_value = list(map(_postprocess_location, other_submission.location.values))
print(other_submission.location.values)
print(postprocess_other_value)

In [ ]:
list_wbf = list(map(_custom_weighted_box_fusion, 
                    postprocess_base_value, 
                    postprocess_other_value))

#no_duplicates_wbf = list(map(_check_duplications, list_wbf))

submission_df = pd.DataFrame({'id': sample_submission['id'].values, 
                              'final_location': list_wbf, 
                              'base_location': sample_submission.location.values, 
                              'added_location': other_submission.location.values})
submission_df

### Test on real submission values

In [ ]:
sample_submission_2 = sample_submission.copy()
sample_submission_2['location'] = ['696 724', '668 693', '203 217', '70 91', '222 258']

In [ ]:
other_submission_2 = sample_submission.copy()
other_submission_2['location'] = ['697 699;701 704;706 711;713 717;719 724', 
                                '669 671;673 676;678 685;687 693', 
                                '204 208;210 217', 
                                '71 82;84 91;177 183', 
                                '223 226;228 229;231 232;234 235;237 240;242 246;248 249;251 254;256 258'
                               ]


In [ ]:
postprocess_base_value_2 = list(map(_postprocess_location, sample_submission_2.location.values))
postprocess_other_value_2 = list(map(_postprocess_location, other_submission_2.location.values))
list_wbf_2 = list(map(_custom_weighted_box_fusion, 
                    postprocess_base_value_2, 
                    postprocess_other_value_2))

#no_duplicates_wbf = list(map(_check_duplications, list_wbf))

submission_df_2 = pd.DataFrame({'id': sample_submission_2['id'].values, 
                              'final_location': list_wbf_2, 
                              'base_location': sample_submission_2.location.values, 
                              'added_location': other_submission_2.location.values})
submission_df_2